<h1> Unlocking Brazil's Mortality Data: A Guide to Extracting SIM-DATASUS Data with the microdatasus Package in R </h1>

DATASUS is the source of healthcare data from Brazil's Unified Healthcare System (called SUS, from the Portuguese "Sistema Unificado de Saúde"). Its goal is to make health data available, and it succeeds in that regard because, to my knowledge, it is the largest and most complete source of healthcare data of any country in the world.

However, although most data is publicly available, getting the data straight from the official source is complicated. [TABNET](https://datasus.saude.gov.br/informacoes-de-saude-tabnet/) offers a simple way to access data, but it requires a lot of manual work to select, download, and organize the datasets. TABNET is interesting if you choose data in a small context, especially because you can pick which variables you want. In addition, due to the regionalization of SUS to decentralize healthcare services, local entities can also make their data available through their own means; it's not solely a federal responsibility. For instance, São Paulo's municipal health department offers [their own version of TABNET](https://www.prefeitura.sp.gov.br/cidade/secretarias/saude/tabnet/). However, as said, it's a lot of work to extract data from it. An alternative to get the datasets is to access the [DATASUS website's FTP](ftp://ftp.datasus.gov.br) or the [web page](https://datasus.saude.gov.br/transferencia-de-arquivos/) to download the .dbc files and use TabWin, a data tabulation software to turn the .dbc into a .csv file (or another filetype you want) using the auxiliary tabulation files. [Here's](https://www.youtube.com/playlist?list=PLD3EOupG4f2O8XjLkzuZtmFqTRJ5IY2KD) an excellent guide (in Portuguese) for this process on YouTube.

Thankfully, we don't need to get the data straight from the source because there are unofficial APIs to make this process a lot easier. At the moment I write, python's library [PySUS](https://github.com/AlertaDengue/PySUS) doesn't work very well, but R's library [microdatasus](https://github.com/rfsaldanha/microdatasus) is there to save us.

DATASUS has many available systems, such as SIM (Mortality Information System), SIA (Outpatient Information System), SINAN (Notifiable Diseases Information System), and SINASC (Live Birth Information System). Here, I'm downloading data from SIM, but the code could be easily modified for another system (hopefully, it works for all of them, but if it doesn't, TabWin will do the job, however more laborious it'll be).

Note: I'm using an integrated Colab + Google Drive environment for this project, but this could be done similarly with a local Jupyter lab, Jupyter notebook, or even just a regular Python compiler.

In [ ]:
# Install and load necessary packages
install.packages("remotes")
remotes::install_github("danicat/read.dbc")
remotes::install_github("rfsaldanha/microdatasus")
library(microdatasus)
library(googledrive)

# Mount Google Drive
drive_auth(use_oob = TRUE)

In [ ]:
# Define the list of states and years to be downloaded
states <- c("AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO",
            "MA", "MG", "MS", "MT", "PA", "PB", "PE", "PI", "PR",
            "RJ", "RN", "RO", "RR", "RS", "SC", "SE", "SP", "TO")
years <- 1996:2022

# Getting mortality data from SIM (Mortality Information System) for each state and year
for (state in states) {
  for (year in years) {
    # Fetch data from DATASUS
    data <- fetch_datasus(year_start = year, year_end = year, uf = state, information_system = "SIM-DO")

    # The file is saved locally as "data.csv" on the notebook before uploading to google drive
    local_file_path <- "data.csv"
    write.csv(data, local_file_path)

    # Specify the target folder in Google Drive
    drive_folder <- paste("seasonal_mortality_hub/data/sim/original_data/",state, "/", sep="")

    # Upload the local CSV file to the specified folder in Google Drive
    drive_upload(local_file_path, name = paste("DO",state,year,".csv", sep=""), path = drive_folder,
                type = "text/csv", overwrite=TRUE)

  }
}